# Homework of CH11

In [80]:
import polars as pl
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

將 mlxtend 函式進行封裝，使其可以直接支援 Polars 的 DataFrame。

In [81]:
def fpgrowth_pl(data: pl.DataFrame, min_support=0.5, use_colnames=False, *args, **kwargs):
    frequent_itemsets = fpgrowth(data.to_pandas(), min_support=min_support, use_colnames=use_colnames, *args, **kwargs)

    frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(lambda x: list(x))
    return pl.DataFrame(frequent_itemsets)

def association_rules_pl(data: pl.DataFrame, metric="confidence", min_threshold=0.8, *args, **kwargs):
    rules = association_rules(data.to_pandas(), metric=metric, min_threshold=min_threshold, *args, **kwargs)
    rules["antecedents"] = rules["antecedents"].apply(lambda x: list(x))
    rules["consequents"] = rules["consequents"].apply(lambda x: list(x))

    return pl.DataFrame(rules)

讀取資料。

In [82]:
df = pl.read_csv("Groceries_dataset.csv")
df

Member_number,Date,itemDescription,Quality
i64,str,str,i64
1808,"""21-07-2015""","""tropical fruit""",1
2552,"""05-01-2015""","""whole milk""",1
2300,"""19-09-2015""","""pip fruit""",1
1187,"""12-12-2015""","""other vegetables""",1
3037,"""01-02-2015""","""whole milk""",1
…,…,…,…
4471,"""08-10-2014""","""sliced cheese""",1
2022,"""23-02-2014""","""candy""",1
1097,"""16-04-2014""","""cake bar""",1


In [83]:
df.describe()

statistic,Member_number,Date,itemDescription,Quality
str,f64,str,str,f64
"""count""",38765.0,"""38765""","""38765""",38765.0
"""null_count""",0.0,"""0""","""0""",0.0
"""mean""",3003.641868,null,null,1.0
"""std""",1153.611031,null,null,0.0
"""min""",1000.0,"""01-01-2014""","""Instant food products""",1.0
"""25%""",2002.0,null,null,1.0
"""50%""",3005.0,null,null,1.0
"""75%""",4007.0,null,null,1.0
"""max""",5000.0,"""31-10-2015""","""zwieback""",1.0


看起來不需要處理空值問題。不過 `itemDescription` 欄位的資料型態是 `str`，需要轉換成 `list`。

In [84]:
df = df.group_by("Member_number").agg(pl.col("itemDescription"))
df

Member_number,itemDescription
i64,list[str]
2373,"[""newspapers"", ""chewing gum""]"
4213,"[""hamburger meat"", ""pork"", … ""seasonal products""]"
4728,"[""other vegetables"", ""beef"", … ""margarine""]"
3427,"[""curd"", ""rolls/buns"", … ""red/blush wine""]"
2114,"[""whole milk"", ""frankfurter"", … ""bottled water""]"
…,…
4695,"[""pork"", ""root vegetables"", … ""shopping bags""]"
3516,"[""photo/film"", ""long life bakery product"", … ""bottled beer""]"
1694,"[""tropical fruit"", ""rolls/buns"", … ""long life bakery product""]"


## Question 1

請使用 `Groceries_dataset` 資料集，並計算銷售資料 `support>=0.1`, `lift>=1` 以上的產品組合

- 提示: 先算出 `support>=0.1` 的 `frequent_itemsets`；再篩選 `lift>=1` 的產品組合

In [85]:
te = TransactionEncoder()
te_ary=te.fit_transform(X=df['itemDescription'])

data_items_dumm = pl.DataFrame(te_ary, schema=te.columns_)
data_items_dumm

Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,…,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,…,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,…,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false
false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,…,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false
false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,…,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false
false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,…,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false
false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,…,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
false,false,false,false,false,false,false,false,true,false,true,false,false,true,false,true,false,false,false,false,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,…,false,false,false,true,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,f

In [86]:
frequent_itemsets = fpgrowth_pl(data_items_dumm, min_support=0.1, use_colnames=True)
frequent_itemsets

support,itemsets
f64,list[str]
0.139815,"[""newspapers""]"
0.458184,"[""whole milk""]"
0.376603,"[""other vegetables""]"
0.349666,"[""rolls/buns""]"
0.132376,"[""pork""]"
…,…
0.15059,"[""whole milk"", ""yogurt""]"
0.112365,"[""whole milk"", ""bottled water""]"
0.113135,"[""whole milk"", ""root vegetables""]"


In [87]:
association_rules_df = association_rules_pl(frequent_itemsets, metric='lift', min_threshold=1)
association_rules_df

antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
list[str],list[str],f64,f64,f64,f64,f64,f64,f64,f64
"[""whole milk""]","[""other vegetables""]",0.458184,0.376603,0.19138,0.417693,1.109106,0.018827,1.070564,0.181562
"[""other vegetables""]","[""whole milk""]",0.376603,0.458184,0.19138,0.508174,1.109106,0.018827,1.101643,0.157802
"[""whole milk""]","[""rolls/buns""]",0.458184,0.349666,0.178553,0.389698,1.114484,0.018342,1.065592,0.189591
"[""rolls/buns""]","[""whole milk""]",0.349666,0.458184,0.178553,0.510638,1.114484,0.018342,1.10719,0.157955
"[""other vegetables""]","[""rolls/buns""]",0.376603,0.349666,0.146742,0.389646,1.114335,0.015056,1.065502,0.164589
…,…,…,…,…,…,…,…,…,…
"[""root vegetables""]","[""whole milk""]",0.230631,0.458184,0.113135,0.490545,1.07063,0.007464,1.063522,0.085746
"[""whole milk""]","[""sausage""]",0.458184,0.206003,0.106978,0.233483,1.133394,0.012591,1.03585,0.217222
"[""sausage""]","[""whole milk""]",0.206003,0.458184,0.106978,0.519303,1.133394,0.012591,1.127146,0.14823
